In [1]:
# Librería para el manejo de datos
import pandas as pd
import numpy as np


import re
import unicodedata


In [21]:
# Caragar datos .csv
df = pd.read_csv('../File/df_corpus_linguistico.csv', sep='\t', encoding='utf-8', index_col=0)
df.head()

,source,target,file_name,quantity_words_source,quantity_words_target,sentiment,NOUN_source,NOUN_target,ADJ_source,ADJ_target,...,VERB_source,VERB_target,VAUX_source,VAUX_target,PRON_source,PRON_target,ADP_source,ADP_target,PUNCT_source,PUNCT_target
id,,,,,,,,,,,,,,,,,,,,,
1,A country whose people are not even aware that...,Un país cuyo pueblo ni siquiera es consciente ...,quora_en_es,29,25,positivo,3.0,6.0,2.0,3.0,...,2.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,3.0
3,! Much Easier to Control the Spread.!,¡! Es mucho más fácil controlar la propagación.!,quora_en_es,7,8,positivo,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0
4,! The Administration can of course clamp Lockd...,"¡! La Administración puede, por supuesto, suje...",quora_en_es,12,13,positivo,3.0,3.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,5.0
5,"!!! WE DO’NT YET KNOW, THE DEGREE IF EFFICATIO...","¡¡¡!!! NO SABEMOS TODAVÍA, EL GRADO DE EFICACIA.",quora_en_es,10,8,neutral,0.0,1.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,8.0
6,!!! You became its next victim when you entere...,¡¡¡!!! Te convertiste en su próxima víctima cu...,quora_en_es,19,19,negativo,3.0,3.0,2.0,2.0,...,3.0,3.0,1.0,0.0,5.0,4.0,0.0,2.0,4.0,7.0


# Columnas importantes para modelar

In [22]:
# Solo dejar columnas source y target
df = df[['source', 'target']]
df.head()

,source,target
id,,
1,A country whose people are not even aware that...,Un país cuyo pueblo ni siquiera es consciente ...
3,! Much Easier to Control the Spread.!,¡! Es mucho más fácil controlar la propagación.!
4,! The Administration can of course clamp Lockd...,"¡! La Administración puede, por supuesto, suje..."
5,"!!! WE DO’NT YET KNOW, THE DEGREE IF EFFICATIO...","¡¡¡!!! NO SABEMOS TODAVÍA, EL GRADO DE EFICACIA."
6,!!! You became its next victim when you entere...,¡¡¡!!! Te convertiste en su próxima víctima cu...


In [23]:
df.shape

(10068314, 2)

# Verificaciones

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10068314 entries, 1 to 12148000
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   source  object
 1   target  object
dtypes: object(2)
memory usage: 230.4+ MB


### 1. Valores nulos

In [25]:
# Valores nulos

df.isnull().sum()

source    0
target    0
dtype: int64

### 2. Valores duplicados

In [26]:
# Verifica si existen frases duplicadas en el corpus de texto 

df.duplicated().sum()

0

### 3. Frases en el target escritas en inglés

In [28]:
# Verifica si existen frases en la columna target sean igual que en la columna source.

df[df.apply(lambda x: x['target'] in x['source'], axis=1)]

,source,target
id,,
118,#jain #jainmuni #spreadpeacelove,#jain #jainmuni #spreadpeacelove
123,#rbe #birdsofaustralia #mynikonlife #abcmyphot...,#rbe #birdsofaustralia #mynikonlife #abcmyphot...
126,#saveindia#impose national emergency#indianarmy,#saveindia#impose national emergency#indianarmy
130,#vitaminD #COVID-19 #coronavirus,#vitaminD #COVID-19 #coronavirus
138,"$1200, $1400, $2800.","$1200, $1400, $2800."
...,...,...
12139929,Atypical meiosis can be adaptive in outcrossed...,Atypical meiosis can be adaptive in outcrossed...
12141378,Autoriserte trafikkskolers landsforbund (ATL) ...,Autoriserte trafikkskolers landsforbund (ATL) ...
12142130,Avomeen Analytical Services Inc.,Avomeen Analytical Services Inc.


### 4. Tratamiento de hashtags y menciones

In [31]:
# Verifica si no hay hashtags, menciones, etc en el texto de origen y destino 

print(f"Hashtags en source: {df['source'].str.contains('#').value_counts()}")

print("-------------------------------------------------------------")

print(f"Hashtags en target: {df['target'].str.contains('#').value_counts()}")

print("-------------------------------------------------------------")

print(f"Menciones en source: {df['source'].str.contains('@').value_counts()}")

print("-------------------------------------------------------------")

print(f"Menciones en target: {df['target'].str.contains('@').value_counts()}")


print("-------------------------------------------------------------")

# Frase con hashtag en el texto de origen y destino

print(f"Hashtag en source: {df[df['source'].str.contains('#')].iloc[0]['source']}")
print(f"Hashtag en target: {df[df['target'].str.contains('#')].iloc[0]['target']}")

# Frase con mención en el texto de origen y destino

print(f"Mención en source: {df[df['source'].str.contains('@')].iloc[0]['source']}")
print(f"Mención en target: {df[df['target'].str.contains('@')].iloc[0]['target']}")

Hashtags en source: source
False    10038602
True        29712
Name: count, dtype: int64
-------------------------------------------------------------
Hashtags en target: target
False    10041773
True        26541
Name: count, dtype: int64
-------------------------------------------------------------
Menciones en source: source
False    9716617
True      351697
Name: count, dtype: int64
-------------------------------------------------------------
Menciones en target: target
False    9720934
True      347380
Name: count, dtype: int64
-------------------------------------------------------------
Hashtag en source: # 2 The HEALTH INDUSTRY, yes they agree .
Hashtag en target: # 2 La INDUSTRIA DE LA SALUD, sí están de acuerdo .
Mención en source: -Amit Singh | instahandle @amitgasm
Mención en target: -Amit Singh | instahandle @amitgasm


### 5. Tratamiento de links

In [32]:
# Frases con links en el texto de origen y destino 

print(f"Links en source: {df[df['source'].str.contains('http')].iloc[0]['source']}")
print(f"Links en target: {df[df['target'].str.contains('http')].iloc[0]['target']}")
print(f"Links en source: {df[df['source'].str.contains('https')].iloc[1]['source']}")
print(f"Links en target: {df[df['target'].str.contains('https')].iloc[1]['target']}")
print(f"Links en source: {df[df['source'].str.contains('www')].iloc[2]['source']}")
print(f"Links en target: {df[df['target'].str.contains('www')].iloc[2]['target']}")

Links en source: AZD1222 Phase-III trial [ http://AZD1222 ]: Ages 18 and up (no age limit)
Links en target: Ensayo AZD1222 Fase-III [ http://AZD1222 ]: A partir de los 18 años (sin límite de edad)
Links en source: Access a Zacks stock report on KFY at https://www.
Links en target: Acceda a un informe de acciones de Zacks en KFY en https: // www.
Links en source: All of the above wordage and terms are extracted (as a synopsis) from the listed article: www.eurkalert.org/pub_releases/2020–04/uoc.cgn040820.php .
Links en target: Todas las palabras y los términos anteriores se han extraído (a modo de sinopsis) del artículo mencionado: www.eurkalert.org/pub_releases/2020-04/uoc.cgn040820.php .


### 6. Verificar presencia de números en las frases

In [33]:
df[df['source'].apply(lambda x: any([token.isdigit() for token in x]))]

,source,target
id,,
14,"!32 countries with new cases and deaths, shoul...","Los 32 países con nuevos casos y muertes, debe..."
16,«1941 - lessons and conclusions».,"'1941 - lecciones y conclusiones""."
17,«1941: Battles on the Baltic 22 June - 10 July...,'1941: Batallas en el Báltico 22 de junio - 10...
18,«50 years of the armed forces of the USSR».,"'50 años de las fuerzas armadas de la URSS""."
20,«Combat and numerical composition of the armed...,'Combate y composición numérica de las fuerzas...
...,...,...
12147876,Because he has only been in parliament since 2...,"Porque solo lleva en el parlamento desde 2015,..."
12147880,"Because he is a juvenile, the name of the 17-y...","Porque es menor de edad, no se dio el nombre d..."
12147886,"Because he played in fewer than four games, he...","Debido a que jugó en menos de cuatro partidos,..."


### 7. Verificar presencia de caracteres especiales en las frases

In [35]:
# Verifica si existen caracteres especiales en el texto de origen y destino

print(f"Caracteres especiales en source: {df['source'].str.contains('[^a-zA-Z.!?]').value_counts()}")

print("-------------------------------------------------------------")

print(f"Caracteres especiales en target: {df['target'].str.contains('[^a-zA-Z.!?]').value_counts()}")

Caracteres especiales en source: source
True    10068314
Name: count, dtype: int64
-------------------------------------------------------------
Caracteres especiales en target: target
True     10065962
False        2352
Name: count, dtype: int64


## **Eliminar frases que sean iguales en inglés como en español**

In [36]:
# Eliminar frases donde al menos tenga 3 palabfras iguales en source y target

df = df[df.apply(lambda x: len(set(x['source'].split()) & set(x['target'].split())) < 3, axis=1)]       

df.shape

(8876079, 2)

Solo entrenaremos el modelo con una parte del dataset

In [37]:
# Obtiene dataset de 10 000 registros
df = df.sample(n=100000, random_state=1)
df.shape

(100000, 2)

In [38]:
df.head()

,source,target
id,,
10151431,I always say that one of the most important th...,Siempre digo que una de las cosas más importan...
5014881,Francis spent every night when he was outside ...,Francisco pasó todas las noches que estuvo fue...
9998244,Find Google- remove your Gmail account.,Encontrar Google- eliminar su cuenta de Gmail.
11709695,"Anyways, apart from that the movie was finalized.","De todos modos, aparte de eso, la película fue..."
5267593,Graphics Programming Black Book and the mindse...,El Libro Negro de la Programación Gráfica y la...


## Funciones de limpieza de datos

In [39]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)      # Normaliza el texto, separando los acentos de las letras
        if unicodedata.category(c) != 'Mn')   

In [40]:
def preprocess_sentence(s):
    s = unicode_to_ascii(s.lower())                 # Convierte a minúsculas y elimina acentos
    s = re.sub(r'@\w+', '<user>', s)                # Reemplaza @username por <user>
    s = re.sub(r'#\w+', '<hashtags>', s)            # Reemplaza #hashtag por <hashtags>
    s = re.sub(r'http\S+', '<url>', s)              # Reemplaza las urls por <url>
    s = re.sub(r'https\S+', '<url>', s)         
    s = re.sub(r'www\S+', '<url>', s)
    s = re.sub(r'([!.?])', r' \1', s)               # Adiciona espacio antes de !, ., ? 
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)           # Elimina caracteres especiales y números
    s = re.sub(r'\s+', r' ', s)                     # Elimina espacios en blanco

    s = s.strip()                                   # Elimina espacios en blanco al inicio y al final          
    s = '<start>' + ' ' + s + ' ' + ' <end>'        # Agrega <start> y <end> a la oración
    return s  

### Aplicar función de limpieza al dataset

In [41]:
# Columna source del dataframe

df['preprocess_source'] = df['source'].apply(preprocess_sentence)

In [42]:
def preprocess_sentence_s(s):
    s = unicode_to_ascii(s.lower())                 # Convierte a minúsculas y elimina acentos
    s = re.sub(r'@\w+', '<usuario>', s)                # Reemplaza @username por <user>
    s = re.sub(r'#\w+', '<hashtags>', s)            # Reemplaza #hashtag por <hashtags>
    s = re.sub(r'http\S+', '<url>', s)
    s = re.sub(r'https\S+', '<url>', s)
    s = re.sub(r'www\S+', '<url>', s)
    s = re.sub(r'([!.?])', r' \1', s)               # Adiciona espacio antes de !, ., ? 
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)           # Elimina caracteres especiales y números
    s = re.sub(r'\s+', r' ', s)                     # Elimina espacios en blanco

    s = s.strip()                                   # Elimina espacios en blanco al inicio y al final          
    s = '<start>' + ' ' + s + ' ' + ' <end>'        # Agrega <start> y <end> a la oración
    return s  

In [44]:
# Aplicar la función preprocess_sentence a la columna source del dataframe
df['preprocess_target'] = df['target'].apply(preprocess_sentence_s)

In [45]:
df.head()

,source,target,preprocess_source,preprocess_target
id,,,,
10151431,I always say that one of the most important th...,Siempre digo que una de las cosas más importan...,<start> i always say that one of the most impo...,<start> siempre digo que una de las cosas mas ...
5014881,Francis spent every night when he was outside ...,Francisco pasó todas las noches que estuvo fue...,<start> francis spent every night when he was ...,<start> francisco paso todas las noches que es...
9998244,Find Google- remove your Gmail account.,Encontrar Google- eliminar su cuenta de Gmail.,<start> find google remove your gmail account ...,<start> encontrar google eliminar su cuenta de...
11709695,"Anyways, apart from that the movie was finalized.","De todos modos, aparte de eso, la película fue...",<start> anyways apart from that the movie was ...,<start> de todos modos aparte de eso la pelicu...
5267593,Graphics Programming Black Book and the mindse...,El Libro Negro de la Programación Gráfica y la...,<start> graphics programming black book and th...,<start> el libro negro de la programacion graf...


In [38]:
# Guardar el dataframe en un archivo .csv
df.to_csv('../File/df_corpus_linguistico_preprocesado.csv', sep='\t', encoding='utf-8')